# Тестирование модели Bert4Rec

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Проверка сервисного блока для загрузки данных о банковских продуктах.

#### Запускаем первые две ячейки данного ноутбука. 
#### Примечание. В скрипте зафиксирован путь к локальным данным data_path, нужно изменить этот путь, куда был загружен набор данных (RecSystemFinal/Bert4Rec/Data/ - желательно, чтобы вторая часть пути оставалось такой). Также важно тут указать k_labels и min_uc.

In [4]:
name_file = 'santander'
data_path = 'C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\' # нужно поменять на свой путь
init_file = 'train_ver2.csv'
k_labels = 3 #указываем количество категорий, которые нужно будет предсказать
min_uc = 10 #если изменили k_labels, то min_uc >= 2*k_labels + k_labels
seed = 42
bert_max_len = 100
bert_mask_prob = 0.15
train_negative_sample_size = 0
train_negative_sampling_seed = 0
test_negative_sample_size = 4
test_negative_sampling_seed = 98765
train_batch_size = 64
val_batch_size = 64
test_batch_size = 64
num_workers = 0
pin_memory = False

In [5]:
from src.datamodules.base_datamodule import BertDataModule
bert_data = BertDataModule(name_file, data_path, init_file, k_labels,
                           min_uc, seed, bert_max_len, bert_mask_prob,
                           train_negative_sample_size, train_negative_sampling_seed,
                           test_negative_sample_size, test_negative_sampling_seed, 
                           train_batch_size, val_batch_size, test_batch_size,
                           num_workers, pin_memory)
bert_data.prepare_data()
bert_data.setup()
train_DL = bert_data.train_dataloader()

Path init ---  C:\Users\boeva\RecSystemFinal\Bert4Rec\Data\train_ver2.csv
Start ...
Creating new path for data ---  C:\Users\boeva\RecSystemFinal\Bert4Rec\Data/santander
Success to create santander.dat
              uid   sid  rating     timestamp
0         15889.0  18.0     1.0  1.432771e+09
1         15889.0  18.0     1.0  1.451261e+09
2         15889.0  18.0     1.0  1.459123e+09
3         15889.0  18.0     1.0  1.464394e+09
4         15891.0   2.0     1.0  1.438042e+09
...           ...   ...     ...           ...
669124  1553541.0   2.0     1.0  1.464394e+09
669125  1553552.0   6.0     1.0  1.464394e+09
669126  1553559.0   2.0     1.0  1.464394e+09
669127  1553565.0   2.0     1.0  1.464394e+09
669128  1553571.0   2.0     1.0  1.464394e+09

[669129 rows x 4 columns]
Turning into implicit ratings
Filtering triplets
10
Densifying index
Splitting
uid
0                    [19, 9, 2, 2, 20, 20, 19, 18, 2, 18]
1       [18, 19, 18, 19, 19, 18, 20, 19, 20, 18, 19, 1...
2                 [1

100%|███████████████████████████████████████████████████████████████████████████| 3899/3899 [00:00<00:00, 21105.30it/s]


random 4 98765
Negative samples don't exist. Generating.
Sampling negative items


100%|███████████████████████████████████████████████████████████████████████████| 3899/3899 [00:00<00:00, 11200.02it/s]


#### Тут представлен успешно отработанный код, который преобразует данные в последовательности событий для каждого клиента.

## Проверка корректности обучения и валидации алгоритмического блока для предсказания рекомендаций банковских продуктов для каждого пользователя.

#### В данном пункте работаем с параметрами по умолчанию. То есть предсказание 3-ех категорий для пользователя. Чтобы данный блок заработал нужно запустить код ниже, добавив такие параметры: model.path_to_res=”...”  datamodule.data_path=’’...” datamodule.init_file=”...” - здесь нужно указать соответствующие пути.

In [ ]:
%run train.py --config-name=train_bert.yaml trainer.gpus=0 trainer.max_epochs=30 datamodule.k_labels=3 model.k_labels=3 datamodule.min_uc=10

[2023-11-13 20:53:59,487][src.utils][INFO] - Disabling python warnings! <config.ignore_warnings=True>
[2023-11-13 20:53:59,489][src.utils][INFO] - Printing config tree with Rich! <config.print_config=True>


CONFIG
├── datamodule
│   └── _target_: src.datamodules.base_datamodule.BertDataModule                                                   
│       name_file: santander                                                                                       
│       data_path: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\                                              
│       init_file: train_ver2.csv                                                                                  
│       k_labels: 3                                                                                                
│       min_uc: 10                                                                                                 
│       seed: 42                                                                                                   
│       bert_max_len: 100                                                                                          
│       bert_mask_prob: 0.15                                                                                       
│       train_negative_sample_size: 0                                                                              
│       train_negative_sampling_seed: 0                                                                            
│       test_negative_sample_size: 4                                                                               
│       test_negative_sampling_seed: 98765                                                                         
│       train_batch_size: 64                                                                                       
│       val_batch_size: 64                                                                                         
│       test_batch_size: 64                                                                                        
│       num_workers: 0                                                                                             
│       pin_memory: false                                                                                          
│                                                                                                                  
├── model
│   └── _target_: src.models.bert.BERTModel                                                                        
│       model_init_seed: 0                                                                                         
│       bert_max_len: 100                                                                                          
│       num_items: 23                                                                                              
│       bert_hidden_units: 256                                                                                     
│       bert_num_blocks: 2                                                                                         
│       bert_num_heads: 4                                                                                          
│       bert_dropout: 0.1                                                                                          
│       lr: 0.003                                                                                                  
│       weight_decay: 0                                                                                            
│       cand_need: false                                                                                           
│       k_labels: 3                                                                                                
│       path_to_res: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\                                
│                                                                                                                  
├── callbacks
│   └── model_checkpoint:                                                                                          
│         

CONFIG
+-- datamodule
|   `-- _target_: src.datamodules.base_datamodule.BertDataModule                                                   
|       name_file: santander                                                                                       
|       data_path: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\                                              
|       init_file: train_ver2.csv                                                                                  
|       k_labels: 3                                                                                                
|       min_uc: 10                                                                                                 
|       seed: 42                                                                                                   
|       bert_max_len: 100                                                                                          
|       bert_mask_prob: 0.15                                                                                       
|       train_negative_sample_size: 0                                                                              
|       train_negative_sampling_seed: 0                                                                            
|       test_negative_sample_size: 4                                                                               
|       test_negative_sampling_seed: 98765                                                                         
|       train_batch_size: 64                                                                                       
|       val_batch_size: 64                                                                                         
|       test_batch_size: 64                                                                                        
|       num_workers: 0                                                                                             
|       pin_memory: false                                                                                          
|                                                                                                                  
+-- model
|   `-- _target_: src.models.bert.BERTModel                                                                        
|       model_init_seed: 0                                                                                         
|       bert_max_len: 100                                                                                          
|       num_items: 23                                                                                              
|       bert_hidden_units: 256                                                                                     
|       bert_num_blocks: 2                                                                                         
|       bert_num_heads: 4                                                                                          
|       bert_dropout: 0.1                                                                                          
|       lr: 0.003                                                                                                  
|       weight_decay: 0                                                                                            
|       cand_need: false                                                                                           
|       k_labels: 3                                                                                                
|       path_to_res: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\                                
|                                                                                                                  
+-- callbacks
|   `-- model_checkpoint:                                                                                          
|         

[2023-11-13 20:53:59,563][src.training_pipeline][INFO] - Instantiating datamodule <src.datamodules.base_datamodule.BertDataModule>
[2023-11-13 20:53:59,659][numexpr.utils][INFO] - NumExpr defaulting to 8 threads.
Already create santander.dat
              uid   sid  rating     timestamp
0         15889.0  18.0     1.0  1.432771e+09
1         15889.0  18.0     1.0  1.451261e+09
2         15889.0  18.0     1.0  1.459123e+09
3         15889.0  18.0     1.0  1.464394e+09
4         15891.0   2.0     1.0  1.438042e+09
...           ...   ...     ...           ...
669124  1553541.0   2.0     1.0  1.464394e+09
669125  1553552.0   6.0     1.0  1.464394e+09
669126  1553559.0   2.0     1.0  1.464394e+09
669127  1553565.0   2.0     1.0  1.464394e+09
669128  1553571.0   2.0     1.0  1.464394e+09

[669129 rows x 4 columns]
Turning into implicit ratings
Filtering triplets
10
Densifying index
Splitting
uid
0                    [19, 9, 2, 2, 20, 20, 19, 18, 2, 18]
1       [18, 19, 18, 19, 19, 18, 20, 1

100%|███████████████████████████████████████████████████████████████████████████| 3899/3899 [00:00<00:00, 17436.89it/s]


...

[2023-11-13 21:55:36,095][src.training_pipeline][INFO] - Starting testing!
[2023-11-13 21:55:36,099][pytorch_lightning.utilities.distributed][INFO] - Restoring states from the checkpoint path at C:\Users\boeva\RecSystemFinal\Bert4Rec\logs\experiments\runs\best_reproduced\2023-11-13_20-53-54\checkpoints\epoch_000.ckpt
[2023-11-13 21:55:36,127][pytorch_lightning.utilities.distributed][INFO] - Loaded model weights from checkpoint at C:\Users\boeva\RecSystemFinal\Bert4Rec\logs\experiments\runs\best_reproduced\2023-11-13_20-53-54\checkpoints\epoch_000.ckpt


Output()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, tensorboard.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gaechka/bert/9efd5793c867481fb4e12b36e133ca16



--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

#### Код отработал без ошибок. Тут представлен вывод основных параметров модели, включена предобработка данных, обучение модели. Лучшая версия модели должны появиться по выведенному пути.
#### Примечание.Также datamodule.k_labels по умолчанию указан 3, но можно указать другое значение, но тогда и значение min_uc должно быть >= 2*k_labels + k_labels - 1. Также нужно удалить папку /preprocessed, чтобы заново запустился datamodule.
#### Код обучения с 5-ю категориями представлен ниже для примера. Тестирование будет проводиться для 3-ех!

#### Предсказание 5-ти категорий.

In [ ]:
%run train.py --config-name=train_bert.yaml trainer.gpus=0 trainer.max_epochs=30 datamodule.k_labels=5 datamodule.min_uc=15 model.k_labels=5

[2023-11-09 01:04:24,044][src.utils][INFO] - Disabling python warnings! <config.ignore_warnings=True>
[2023-11-09 01:04:24,047][src.utils][INFO] - Printing config tree with Rich! <config.print_config=True>


CONFIG
├── datamodule
│   └── _target_: src.datamodules.base_datamodule.BertDataModule                                                   
│       name_file: santander                                                                                       
│       data_path: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\                                              
│       init_file: train_ver2.csv                                                                                  
│       k_labels: 5                                                                                                
│       min_uc: 15                                                                                                 
│       seed: 42                                                                                                   
│       bert_max_len: 100                                                                                          
│       bert_mask_prob: 0.15                                                                                       
│       train_negative_sample_size: 0                                                                              
│       train_negative_sampling_seed: 0                                                                            
│       test_negative_sample_size: 4                                                                               
│       test_negative_sampling_seed: 98765                                                                         
│       train_batch_size: 64                                                                                       
│       val_batch_size: 64                                                                                         
│       test_batch_size: 64                                                                                        
│       num_workers: 0                                                                                             
│       pin_memory: false                                                                                          
│                                                                                                                  
├── model
│   └── _target_: src.models.bert.BERTModel                                                                        
│       model_init_seed: 0                                                                                         
│       bert_max_len: 100                                                                                          
│       num_items: 23                                                                                              
│       bert_hidden_units: 256                                                                                     
│       bert_num_blocks: 2                                                                                         
│       bert_num_heads: 4                                                                                          
│       bert_dropout: 0.1                                                                                          
│       lr: 0.003                                                                                                  
│       weight_decay: 0                                                                                            
│       cand_need: false                                                                                           
│       k_labels: 5                                                                                                
│       path_to_res: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\                                
│                                                                                                                  
├── callbacks
│   └── model_checkpoint:                                                                                          
│         

CONFIG
+-- datamodule
|   `-- _target_: src.datamodules.base_datamodule.BertDataModule                                                   
|       name_file: santander                                                                                       
|       data_path: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\                                              
|       init_file: train_ver2.csv                                                                                  
|       k_labels: 5                                                                                                
|       min_uc: 15                                                                                                 
|       seed: 42                                                                                                   
|       bert_max_len: 100                                                                                          
|       bert_mask_prob: 0.15                                                                                       
|       train_negative_sample_size: 0                                                                              
|       train_negative_sampling_seed: 0                                                                            
|       test_negative_sample_size: 4                                                                               
|       test_negative_sampling_seed: 98765                                                                         
|       train_batch_size: 64                                                                                       
|       val_batch_size: 64                                                                                         
|       test_batch_size: 64                                                                                        
|       num_workers: 0                                                                                             
|       pin_memory: false                                                                                          
|                                                                                                                  
+-- model
|   `-- _target_: src.models.bert.BERTModel                                                                        
|       model_init_seed: 0                                                                                         
|       bert_max_len: 100                                                                                          
|       num_items: 23                                                                                              
|       bert_hidden_units: 256                                                                                     
|       bert_num_blocks: 2                                                                                         
|       bert_num_heads: 4                                                                                          
|       bert_dropout: 0.1                                                                                          
|       lr: 0.003                                                                                                  
|       weight_decay: 0                                                                                            
|       cand_need: false                                                                                           
|       k_labels: 5                                                                                                
|       path_to_res: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\                                
|                                                                                                                  
+-- callbacks
|   `-- model_checkpoint:                                                                                          
|         

[2023-11-09 01:04:24,101][src.training_pipeline][INFO] - Instantiating datamodule <src.datamodules.base_datamodule.BertDataModule>
Already create santander.dat
Already preprocessed. Skip preprocessing
random 0 0
Negatives samples exist. Loading.
random 4 98765
Negatives samples exist. Loading.
[2023-11-09 01:04:24,114][src.training_pipeline][INFO] - Instantiating model <src.models.bert.BERTModel>
[2023-11-09 01:04:24,137][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
[2023-11-09 01:04:24,141][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
[2023-11-09 01:04:24,144][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichModelSummary>
[2023-11-09 01:04:24,146][src.training_pipeline][INFO] - Instantiating callback <pytorch_lightning.callbacks.RichProgressBar>
[2023-11-09 01:04:24,147][src.training_pipeline][INFO] - Instantiating logger <pytorch_lightning.

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorboard, torch.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gaechka/bert/9e1505ece661464e89375c3cd5ab7c73
COMET INFO:   Others:
COMET INFO:     Name : bert
COMET INFO:   Parameters:
COMET INFO:     callbacks/early_stopping/_target_                  : pytorch_lightning.callbacks.EarlyStopping
COMET INFO:     callbacks/early_stopping/min_delta                 : 0
COMET INFO:     callbacks/early_stopping/mode                      : max
COMET INFO:     callbacks/early_stopping/monitor                   : val/Recall@3
COMET INFO:     callbacks/early_stopping/patience         

...

[2023-11-09 01:04:52,166][src.training_pipeline][INFO] - Starting testing!
[2023-11-09 01:04:52,172][pytorch_lightning.utilities.distributed][INFO] - Restoring states from the checkpoint path at C:\Users\boeva\RecSystemFinal\Bert4Rec\logs\experiments\runs\best_reproduced\2023-11-09_01-04-23\checkpoints\epoch_000.ckpt
[2023-11-09 01:04:52,217][pytorch_lightning.utilities.distributed][INFO] - Loaded model weights from checkpoint at C:\Users\boeva\RecSystemFinal\Bert4Rec\logs\experiments\runs\best_reproduced\2023-11-09_01-04-23\checkpoints\epoch_000.ckpt


Output()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorboard, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gaechka/bert/a807fc89f206428ea1bbb4e97afce34f



--------------------------------------------------------------------------------

DATALOADER:0 TEST RESULTS

## Проверка корректности  вывод рекомендаций для каждого пользователя при окончании работы алгоритмического блока.

#### Запускаем ячейку ниже. Примечание: нужно перекопировать путь до checkpoint из блока, где обучение проводилос, для 3-ех категорий, и вставить его в конфиг test.yaml в переменную ckpt_path или указать через командную строку.

In [7]:
%run test.py trainer.gpus=0 ckpt_path="C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\logs\\experiments\\runs\\best_reproduced\\2023-11-13_20-53-54\\checkpoints\\epoch_000.ckpt" datamodule.k_labels=3 datamodule.min_uc=10 model.k_labels=3

[2023-11-13 22:03:51,923][src.utils][INFO] - Disabling python warnings! <config.ignore_warnings=True>
[2023-11-13 22:03:51,925][src.utils][INFO] - Printing config tree with Rich! <config.print_config=True>
[2023-11-13 22:03:51,926][src.utils][INFO] - Field 'callbacks' not found in config
[2023-11-13 22:03:51,927][src.utils][INFO] - Field 'logger' not found in config


CONFIG
├── datamodule
│   └── _target_: src.datamodules.base_datamodule.BertDataModule                                                   
│       name_file: santander                                                                                       
│       data_path: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\                                              
│       init_file: train_ver2.csv                                                                                  
│       k_labels: 3                                                                                                
│       min_uc: 10                                                                                                 
│       seed: 42                                                                                                   
│       bert_max_len: 100                                                                                          
│       bert_mask_prob: 0.15                                                                                       
│       train_negative_sample_size: 0                                                                              
│       train_negative_sampling_seed: 0                                                                            
│       test_negative_sample_size: 4                                                                               
│       test_negative_sampling_seed: 98765                                                                         
│       train_batch_size: 64                                                                                       
│       val_batch_size: 64                                                                                         
│       test_batch_size: 64                                                                                        
│       num_workers: 0                                                                                             
│       pin_memory: false                                                                                          
│                                                                                                                  
├── model
│   └── _target_: src.models.bert.BERTModel                                                                        
│       model_init_seed: 0                                                                                         
│       bert_max_len: 100                                                                                          
│       num_items: 23                                                                                              
│       bert_hidden_units: 256                                                                                     
│       bert_num_blocks: 2                                                                                         
│       bert_num_heads: 4                                                                                          
│       bert_dropout: 0.1                                                                                          
│       lr: 0.003                                                                                                  
│       weight_decay: 0                                                                                            
│       cand_need: false                                                                                           
│       k_labels: 3                                                                                                
│       path_to_res: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\                                
│                                                                                                                  
├── trainer
│   └── _target_: pytorch_lightning.Trainer                                                                        
│       gpus

CONFIG
+-- datamodule
|   `-- _target_: src.datamodules.base_datamodule.BertDataModule                                                   
|       name_file: santander                                                                                       
|       data_path: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\Data\\                                              
|       init_file: train_ver2.csv                                                                                  
|       k_labels: 3                                                                                                
|       min_uc: 10                                                                                                 
|       seed: 42                                                                                                   
|       bert_max_len: 100                                                                                          
|       bert_mask_prob: 0.15                                                                                       
|       train_negative_sample_size: 0                                                                              
|       train_negative_sampling_seed: 0                                                                            
|       test_negative_sample_size: 4                                                                               
|       test_negative_sampling_seed: 98765                                                                         
|       train_batch_size: 64                                                                                       
|       val_batch_size: 64                                                                                         
|       test_batch_size: 64                                                                                        
|       num_workers: 0                                                                                             
|       pin_memory: false                                                                                          
|                                                                                                                  
+-- model
|   `-- _target_: src.models.bert.BERTModel                                                                        
|       model_init_seed: 0                                                                                         
|       bert_max_len: 100                                                                                          
|       num_items: 23                                                                                              
|       bert_hidden_units: 256                                                                                     
|       bert_num_blocks: 2                                                                                         
|       bert_num_heads: 4                                                                                          
|       bert_dropout: 0.1                                                                                          
|       lr: 0.003                                                                                                  
|       weight_decay: 0                                                                                            
|       cand_need: false                                                                                           
|       k_labels: 3                                                                                                
|       path_to_res: C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\                                
|                                                                                                                  
+-- trainer
|   `-- _target_: pytorch_lightning.Trainer                                                                        
|       gpus

[2023-11-13 22:03:51,974][src.testing_pipeline][INFO] - Instantiating datamodule <src.datamodules.base_datamodule.BertDataModule>
Already create santander.dat
Already preprocessed. Skip preprocessing
random 0 0
Negatives samples exist. Loading.
random 4 98765
Negatives samples exist. Loading.
[2023-11-13 22:03:51,995][src.testing_pipeline][INFO] - Instantiating model <src.models.bert.BERTModel>
[2023-11-13 22:03:52,017][src.testing_pipeline][INFO] - Instantiating trainer <pytorch_lightning.Trainer>
[2023-11-13 22:03:52,025][pytorch_lightning.utilities.distributed][INFO] - GPU available: False, used: False
[2023-11-13 22:03:52,027][pytorch_lightning.utilities.distributed][INFO] - TPU available: False, using: 0 TPU cores
[2023-11-13 22:03:52,029][pytorch_lightning.utilities.distributed][INFO] - IPU available: False, using: 0 IPUs
[2023-11-13 22:03:52,030][src.testing_pipeline][INFO] - Starting testing!
[2023-11-13 22:03:52,034][pytorch_lightning.utilities.distributed][INFO] - Restoring s

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/MRR@1': 0.9794819184406258,
 'test/MRR@2': 0.7581431136188767,
 'test/MRR@3': 0.6457567467441794,
 'test/NDCG@1': 0.9794819355010986,
 'test/NDCG@2': 0.9612255692481995,
 'test/NDCG@3': 0.9158613681793213,
 'test/Precision@1': 0.9794819355010986,
 'test/Precision@3': 0.8932204842567444,
 'test/Recall@1': 0.9794819355010986,
 'test/Recall@2': 0.9558861255645752,
 'test/Recall@3': 0.8932204842567444}
--------------------------------------------------------------------------------


#### Скрипт выводит метрики, полученные путем тестирования алгоритмического блока. Метрики  Recall@3  и NDCG@3 на этапе тестирования должны превышать 0.89.

## Проверка корректности  вывод рекомендаций для каждого пользователя при окончании работы алгоритмического блока.

#### Запускаем следующую ячейку.
#### Примечание: нужно поменять путь к результирующим файлам, который был указан в конфиге model/bert.yaml, это нужно сделать в пункте про обучение выше.

In [8]:
dict_user_label = dict()
with open('C:\\Users\\boeva\\RecSystemFinal\\Bert4Rec\\results\\res_raw\\results_0.txt') as f:
    for lines in f:
        key, value = lines.split()[0], lines.split()[1:]
        dict_user_label[key] = value

In [9]:
dict_user_label

{'user': ['label1', 'label2', 'label3'],
 '0': ['19', '20', '18'],
 '1': ['18', '19', '20'],
 '2': ['18', '19', '20'],
 '3': ['20', '18', '19'],
 '4': ['18', '19', '20'],
 '5': ['19', '2', '20'],
 '6': ['19', '20', '18'],
 '7': ['20', '19', '15'],
 '8': ['19', '18', '2'],
 '9': ['18', '19', '20'],
 '10': ['19', '20', '18'],
 '11': ['18', '19', '20'],
 '12': ['20', '18', '19'],
 '13': ['19', '20', '15'],
 '14': ['18', '20', '19'],
 '15': ['19', '20', '2'],
 '16': ['18', '19', '20'],
 '17': ['19', '18', '20'],
 '18': ['19', '20', '18'],
 '19': ['20', '19', '18'],
 '20': ['20', '18', '19'],
 '21': ['19', '20', '18'],
 '22': ['18', '19', '20'],
 '23': ['20', '19', '15'],
 '24': ['18', '19', '20'],
 '25': ['19', '20', '18'],
 '26': ['20', '19', '18'],
 '27': ['18', '19', '20'],
 '28': ['19', '20', '18'],
 '29': ['20', '19', '15'],
 '30': ['18', '19', '20'],
 '31': ['18', '19', '20'],
 '32': ['19', '18', '20'],
 '33': ['18', '19', '20'],
 '34': ['18', '19', '20'],
 '35': ['20', '19', '18'],


#### Код успешно отработал. Вывод - это словарь предсказанных категорий для пользователей. 